In [ ]:
import numpy as np
import pymc as pm
import pandas as pd
import arviz as az

data = pd.read_csv('Prices.csv')
y = data['Price'].values
x1 = data['Speed'].values
x2 = np.log(data['HardDrive'].values)

# Punctul a
with pm.Model() as model:
    alfa = pm.Normal('alfa', mu=0, sigma=1000)
    beta1 = pm.Normal('beta1', mu=0, sigma=100)
    beta2 = pm.Normal('beta2', mu=0, sigma=10)
    sigma = pm.HalfNormal('sigma', sigma=500)
    
    mu = alfa+ beta1 * x1 + beta2 * x2
    y_pred = pm.Normal('y_pred', mu=mu, sigma=sigma, observed=y)
    idata = pm.sample(1000, tune=1000, random_seed=42, target_accept=0.9)

# Punctul b
beta1_hdi = az.hdi(idata, var_names=['beta1'], hdi_prob=0.95)
beta2_hdi = az.hdi(idata, var_names=['beta2'], hdi_prob=0.95)

print(f"beta1 95% HDI = [{beta1_hdi['beta1'].values[0]:.4f}, {beta1_hdi['beta1'].values[1]:.4f}]")
print(f"beta2 95% HDI = [{beta2_hdi['beta2'].values[0]:.4f}, {beta2_hdi['beta2'].values[1]:.4f}]")

# Punctl d
x1 = 33 
x2 = np.log(540) 

posterior_draws = idata.posterior.stack(samples=("chain", "draw"))
alfa = posterior_draws['alfa'].values
beta1 = posterior_draws['beta1'].values
beta2 = posterior_draws['beta2'].values

mu_new_draws = alfa + beta1 * x1 + beta2 * x2
mu_hdi = az.hdi(mu_new_draws, hdi_prob=0.90)
print(f"90% HDI mu: [{mu_hdi[0]:.4f}, {mu_hdi[1]:.4f}]")

# Punctul e
sigma_samples = posterior_draws['sigma'].values
y_pred_draws = np.random.normal(mu_new_draws, sigma_samples)

y_pred_hdi = az.hdi(y_pred_draws, hdi_prob=0.90)
print(f"90% HDI: [{y_pred_hdi[0]:.4f}, {y_pred_hdi[1]:.4f}]")